In [ ]:
%matplotlib inline

from pathlib import Path
import io
import urllib.parse as urlparse
import string
import random

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tqdm.auto import trange
from tqdm.auto import tqdm
import requests
from scrapy.http import TextResponse
from PIL import Image

model_dir = Path("/home/hadim/.data/Neural_Network/captchanet/model/")

model_name = 'v5'
model_path = model_dir / model_name
tokenizer_path = model_path / 'tokenizer.json'

---
Download random images and run the model on it. Here we don't have access to the true captcha label.

In [ ]:
# Test predictions on randomly downloaded captcha images.

# Load tokenizer and model
with open(model_path / 'tokenizer.json') as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())
model = tf.keras.models.load_model(str(model_path / 'model'))

# Download images
batch_size = 8
captcha_url = "https://www.referendum.interieur.gouv.fr/bundles/ripconsultation/securimage/securimage_show.php"
images = [np.array(Image.open(io.BytesIO(requests.get(captcha_url).content))) for _ in trange(batch_size)]
images = np.array(images)

# Preprocess the image
# The images need to be normalized t [0, 1] and resize to (224, 224).
image_size = (224, 224)
batch = tf.cast(images, 'float32')
batch = tf.image.per_image_standardization(batch)
batch = tf.image.resize(batch, image_size)

# Run inference
labels = model(batch)

# Postprocess results (decode labels)
labels = tf.argmax(labels, axis=2)
words = tokenizer.sequences_to_texts(labels.numpy())
words = [word.replace(' ', '') for word in words]

fig_size = 2
ncols = 2
nrows = batch_size // ncols
ratio = images.shape[2] / images.shape[1]
figsize = (ncols * fig_size * ratio, fig_size * nrows)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, constrained_layout=True)
axs = axs.flatten()

for image, ax, word in zip(images, axs, words):
  ax.imshow(image)
  text = f'Predicted: {word} ({len(word)})'
  ax.set_title(text, fontsize=18)

---

Here we try to solve the captcha by postinf the captcha answer to the website.

In [ ]:
# Load tokenizer and model
with open(model_path / 'tokenizer.json') as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())
model = tf.keras.models.load_model(str(model_path / 'model'))

def captcha_solver(image):
    
  image = np.array(image)
  images = np.array([image])
  
  # The images need to be normalized t [0, 1] and resize to (224, 224).
  image_size = (224, 224)
  batch = tf.cast(images, 'float32')
  batch = tf.image.per_image_standardization(batch)
  batch = tf.image.resize(batch, image_size)

  # Run inference
  labels = model(batch)

  # Postprocess results (decode labels)
  labels = tf.argmax(labels, axis=2)
  words = tokenizer.sequences_to_texts(labels.numpy())
  words = [word.replace(' ', '') for word in words]
  
  return words[0]

In [ ]:
root_url = "https://www.referendum.interieur.gouv.fr/consultation_publique/8/"

n = 500
success = 0
rate = 0
failed_images = []
failed_words = []

for i in trange(n):

  letter1 = random.choice(string.ascii_uppercase)
  letter2 = random.choice(string.ascii_uppercase)
  letter3 = random.choice(string.ascii_uppercase)
  url = urlparse.urljoin(root_url, f'{letter1}/{letter2}{letter3}')

  cookies = {}
  cookies['incap_ses_700_2043128'] = 'z00hQBP2mS515CRSzvK2CarcKV0AAAAA54RCSzqGm0bptUV4OJjvlA=='
  session = requests.Session()
  requests.utils.add_dict_to_cookiejar(session.cookies, cookies)

  req = session.get(url)
  response = TextResponse(str(req.url), body=req.text, encoding='utf-8')
  
  iframe_src = response.css('iframe').xpath('@src').get()
  if iframe_src and 'Incapsula' in iframe_src:
    raise Exception("Incapsula issue.")

  captcha = response.css('img#captcha').xpath('@src').get()

  if not captcha:
    raise Exception("Captcha not here.")

  # Get the token
  token = response.css('#form__token').xpath('@value').get()

  # Get the image captcha URL
  captcha_uri = response.css('img#captcha').xpath('@src').get()
  captcha_url = urlparse.urljoin(root_url, captcha_uri)

  # Download the image
  req = session.get(captcha_url)
  captcha_image = Image.open(io.BytesIO(req.content))

  # Solve the captcha
  captcha_solution = captcha_solver(captcha_image)

  # Send captcha solution.
  form_data = {}
  form_data['form[captcha]'] = captcha_solution
  form_data['form[_token]'] = token

  # Get the actual page.
  req = session.post(url, data=form_data)
  response = TextResponse(str(req.url), body=req.text, encoding='utf-8')
  
  iframe_src = response.css('iframe').xpath('@src').get()
  if iframe_src and 'Incapsula' in iframe_src:
    raise Exception("Incapsula issue.")

  captcha = response.css('img#captcha').xpath('@src').get()

  if not captcha:
    success += 1
  else:
    failed_images.append(np.asarray(captcha_image))
    failed_words.append(np.asarray(captcha_solution))
    
  rate = (success / (i+1)) * 100
  tqdm.write(f"{rate:.1f} ({success}/{i+1})")